# // TO DO LIST//

1. get the function get_cities_website to properly work - get past https request issue
2. build out function to get all dates from football season and save into csv files into directory using OS
3. convert state names in stadium dataframe to using abbreviations using some replace or something function (lambdas)
4. make OS directory list using the following format

    - year
      - city name
          - date

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import pickle
from bs4 import BeautifulSoup
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import lxml

In [5]:
# this function will get a dictionairy of cities with {city: website_url}
def get_cities_website(list_of_cities):
    
    # driver location downloaded from: https://chromedriver.chromium.org/downloads
    
    driver = webdriver.Chrome(r'D:\Downloads\chromedriver_win32\chromedriver.exe')
    
    # will store all cities and their websites
    dict_of_cities = {}
    
    driver.get('https://www.wunderground.com/history')
    for city in list_of_cities:
        
        driver.get('https://www.wunderground.com/history')
        
        time.sleep(1)
        input_city = driver.find_element_by_xpath('//*[@id="historySearch"]')
        input_city.send_keys(city)
        time.sleep(1)
        input_city.send_keys(Keys.ENTER)

        view_button = driver.find_element_by_xpath('//*[@id="dateSelect"]/div/input')
        view_button.click()
        time.sleep(1)
        dict_of_cities[city] = driver.current_url
        
        print('Loaded data from', city, driver.current_url)
        time.sleep(120)
        
      
    # closes the web browser
    driver.close()
    
    return dict_of_cities

In [6]:
# example of how above function works
# will need to parse out date in future method to get rid of current date, then can easily dump to pickle file or csv
print(get_cities_website(['Inglewood, CA', 'Foxborough, MA']))

Loaded data from Inglewood, CA https://www.wunderground.com/history/daily/KLAX/date/2020-4-5
Loaded data from Foxboro, MA https://www.wunderground.com/history


WebDriverException: Message: chrome not reachable
  (Session info: chrome=80.0.3987.149)


In [7]:
# fully functional, however I think using https://geojango.com/pages/list-of-nfl-teams in conjunction with this table will be better

def get_stadium_cities_wiki():
    url = 'https://en.wikipedia.org/wiki/List_of_current_National_Football_League_stadiums'
    table = pd.read_html(url)
    df = table[1]
    
    keep_cols = ['Name', 'Capacity', 'Location', 'Surface', 'Roof type', 'Team(s)', 'Opened']
    
    df = df[keep_cols]
    
    
    
    df['Capacity'] = df['Capacity'].astype(int)
    df['Surface'] = df['Surface'].replace(to_replace=r'\[\d{1,2}\]', value='', regex=True)
    df['Opened'] = df['Opened'].replace(to_replace=r'\[.*?\]', value='', regex=True)
    
    # need to fix this regex to properly divide the column into 2
#     df['Team(s)'] = df['Team(s)'].str.split(r'(.*[A-Za-z]+)([A-Z].*)', expand=True)
        
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_from_wiki.pkl')
    
def get_stadium_cities_nonwiki():
    url = 'https://geojango.com/pages/list-of-nfl-teams'
    table = pd.read_html(url)
    
    df = table[0]
    
    
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_not_from_wiki.pkl')
    
    
def get_state_abbrevs():
    url2 = 'https://www.softschools.com/social_studies/state_abbreviations/'
    table2 = pd.read_html(url2)
    df2 = table2[2]
    df2['State'] = df2['State'].str.title()
    df2.set_index('State', inplace=True)
    pd.to_pickle(df2, './state_abbreviations.pkl')

In [8]:
get_stadium_cities_nonwiki()
get_state_abbrevs()

In [9]:
# load dataframe from pickle file
stadiums_df = pd.read_pickle('./stadium_dataframe_not_from_wiki.pkl')
stadiums_df

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year
0,Arizona Cardinals,State Farm Stadium,"Glendale, Arizona",63400,2006
1,Atlanta Falcons,Mercedes-Benz Stadium,"Atlanta, Georgia",71000,2017
2,Baltimore Ravens,M&T Bank Stadium,"Baltimore, Maryland",71008,1998
3,Buffalo Bills,New Era Field,"Orchard Park, New York",71608,1973
4,Carolina Panthers,Bank of America Stadium,"Charlotte, North Carolina",75523,1996
5,Chicago Bears,Soldier Field,"Chicago, Illinois",61500,1924
6,Cincinnati Bengals,Paul Brown Stadium,"Cincinnati, Ohio",65515,2000
7,Cleveland Browns,First Energy Stadium,"Cleveland, Ohio",67895,1999
8,Dallas Cowboys,At&T Stadium,"Arlington, Texas",80000,2009
9,Denver Broncos,Empower Field at Mile High,"Denver, Colorado",76125,2001


In [10]:
state_abbrevs = pd.read_pickle('./state_abbreviations.pkl')
state_abbrevs.to_dict()['Abbreviation']

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY'}

In [7]:
# get location data based on location of stadiums

city_list = stadiums_df['Arena Location'].tolist()
city_dict = get_cities_website(city_list)
city_dict

Loaded data from Glendale, Arizona
Loaded data from Atlanta, Georgia
Loaded data from Baltimore, Maryland


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51768): Max retries exceeded with url: /session/148a0284573a9e81ee9359cc117922c8/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000200E61395C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))